# Dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib inline

# Preprocessing the Data

In [2]:
# read csv into dataframe
filepath = "static/resources/Austin Properties Dataset.csv"
csv_df = pd.read_csv(filepath)

In [3]:
# check the column names
csv_df.columns

Index(['Address', 'Building Name', 'Zip Code', 'Bed', 'Bath', 'Avg SF',
       '# Units', 'Mix %', 'Units Available Units', 'Units Available Percent',
       'Avg Asking Rent/Unit', 'Avg Asking Rent/SF', 'Avg Effective Rent/Unit',
       'Avg Effective Rent/SF', 'Concessions %', 'Year Built',
       'Year Renovated', 'Units', 'Lande (Acres)', 'Vacancy %', 'Washer/Dryer',
       'Walk in Closet', 'Hardwood/Vinyl Floor', 'Walk Score', 'Transit Score',
       '1 Mile Population', '1 Mile Median Household Income',
       'Miles from Domain', 'Miles from Downtown', 'Unnamed: 29'],
      dtype='object')

In [4]:
csv_df = pd.get_dummies(data=csv_df, columns=['Washer/Dryer'])

In [5]:
# select certain variables
original_df = csv_df[['Bed', 'Bath', 'Avg SF', 'Concessions %', 'Year Built', 
   'Walk in Closet', 'Hardwood/Vinyl Floor',
   'Washer/Dryer_Yes but not in unit', 'Washer/Dryer_Yes in unit', 'Washer/Dryer_no',
   'Walk Score', 'Transit Score', '1 Mile Population', '1 Mile Median Household Income',
   'Miles from Domain', 'Miles from Downtown', 
   'Avg Effective Rent/Unit']]

In [6]:
# process na values
austin_properties_df = original_df.dropna()
# check the result of na value processing
austin_properties_df.count()

Bed                                 5468
Bath                                5468
Avg SF                              5468
Concessions %                       5468
Year Built                          5468
Walk in Closet                      5468
Hardwood/Vinyl Floor                5468
Washer/Dryer_Yes but not in unit    5468
Washer/Dryer_Yes in unit            5468
Washer/Dryer_no                     5468
Walk Score                          5468
Transit Score                       5468
1 Mile Population                   5468
1 Mile Median Household Income      5468
Miles from Domain                   5468
Miles from Downtown                 5468
Avg Effective Rent/Unit             5468
dtype: int64

In [7]:
# check the data type
austin_properties_df.dtypes

Bed                                 float64
Bath                                float64
Avg SF                              float64
Concessions %                       float64
Year Built                          float64
Walk in Closet                      float64
Hardwood/Vinyl Floor                float64
Washer/Dryer_Yes but not in unit      uint8
Washer/Dryer_Yes in unit              uint8
Washer/Dryer_no                       uint8
Walk Score                          float64
Transit Score                       float64
1 Mile Population                   float64
1 Mile Median Household Income      float64
Miles from Domain                   float64
Miles from Downtown                 float64
Avg Effective Rent/Unit             float64
dtype: object

In [8]:
# overview of the new dataframe
austin_properties_df.head()

,Bed,Bath,Avg SF,Concessions %,Year Built,Walk in Closet,Hardwood/Vinyl Floor,Washer/Dryer_Yes but not in unit,Washer/Dryer_Yes in unit,Washer/Dryer_no,Walk Score,Transit Score,1 Mile Population,1 Mile Median Household Income,Miles from Domain,Miles from Downtown,Avg Effective Rent/Unit
0,1.0,1.0,560.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1528.0
1,1.0,1.0,612.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1633.0
2,1.0,1.0,629.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1740.0
3,1.0,1.0,774.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1778.0
4,1.0,1.0,778.0,0.01,2016.0,1.0,1.0,0,1,0,88.0,44.0,20115.0,70385.0,12.6,2.1,1852.0


In [9]:
austin_properties_df["Year Built"].mean()

1997.693672275055

In [10]:
austin_properties_df["1 Mile Population"].mean()

16014.830651060716

# Determine X and y for Machine Learning

In [11]:
X = austin_properties_df.iloc[:, :-1]
y = austin_properties_df['Avg Effective Rent/Unit']

# Splitting the data into training and testing sets

In [12]:
from sklearn.model_selection import train_test_split

# 80/20 training and testing data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [13]:
# check the shape of training and test data
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4374, 16)
(1094, 16)
(4374,)
(1094,)


# Model (I) - Linear Regression

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [15]:
# train the model
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [16]:
# model evaluation
print(f"R-quared (R2): {r2_score(y_test, lin_model.predict(X_test))}")

R-quared (R2): 0.5344241825987113


# Model (II) - Polynomial Regression

In [17]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

In [18]:
poly_model = Pipeline([
    ('a', PolynomialFeatures(degree=2)),
    ('b', LinearRegression(fit_intercept=False))
])

In [19]:
poly_model.fit(X_train, y_train)
poly_model.score(X_test, y_test)

0.44883007398009794

# Model (III) Random Forest
## Please note the actual model is: param_grid = {'n_estimators' : list(range(1,999,2))}

In [20]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

forest = RandomForestRegressor()
param_grid = {'n_estimators' : list(range(1,61,2))}
model_rf = GridSearchCV(forest, param_grid, verbose=3)

model_rf.fit(X_train, y_train)
print(model_rf.best_params_)
print(model_rf.best_score_)

y_pred_rf = model_rf.predict(X_test)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] n_estimators=1 ..................................................
[CV] ......... n_estimators=1, score=0.9004432594857569, total=   0.0s
[CV] n_estimators=1 ..................................................
[CV] ......... n_estimators=1, score=0.8086335972653349, total=   0.0s
[CV] n_estimators=1 ..................................................
[CV] ......... n_estimators=1, score=0.8547785395162539, total=   0.0s
[CV] n_estimators=3 ..................................................


/Users/apple/Documents/anaconda3/envs/PyData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] .......... n_estimators=3, score=0.931731149568489, total=   0.0s
[CV] n_estimators=3 ..................................................
[CV] ......... n_estimators=3, score=0.8674191897953709, total=   0.0s
[CV] n_estimators=3 ..................................................
[CV] ......... n_estimators=3, score=0.9127841313462336, total=   0.0s
[CV] n_estimators=5 ..................................................
[CV] ......... n_estimators=5, score=0.9176215192917629, total=   0.1s
[CV] n_estimators=5 ..................................................
[CV] ......... n_estimators=5, score=0.9115885186924367, total=   0.1s
[CV] n_estimators=5 ..................................................
[CV] ......... n_estimators=5, score=0.8959544298824667, total=   0.1s
[CV] n_estimators=7 ..................................................
[CV] ......... n_estimators=7, score=0.9276798700593154, total=   0.1s
[CV] n_estimators=7 ..................................................
[CV] .

[CV] ........ n_estimators=41, score=0.9252989251592008, total=   0.6s
[CV] n_estimators=41 .................................................
[CV] ........ n_estimators=41, score=0.9300114769159891, total=   0.6s
[CV] n_estimators=43 .................................................
[CV] ........ n_estimators=43, score=0.9422630453290184, total=   0.6s
[CV] n_estimators=43 .................................................
[CV] ........ n_estimators=43, score=0.9363909157810851, total=   0.6s
[CV] n_estimators=43 .................................................
[CV] ........ n_estimators=43, score=0.9290245713025812, total=   0.6s
[CV] n_estimators=45 .................................................
[CV] ........ n_estimators=45, score=0.9409541568618518, total=   0.6s
[CV] n_estimators=45 .................................................
[CV] ........ n_estimators=45, score=0.9227116701398383, total=   0.6s
[CV] n_estimators=45 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:   40.6s finished


{'n_estimators': 37}
0.9367381542339314


In [21]:
# import pickle
# rf_model_filepath = '/Users/apple/Documents/Data Projects/Machine_Learning_Apartment_Rent/static/resources/RF999_Model.sav'
# pickle.dump(model_rf, open(rf_model_filepath, 'wb'))

In [22]:
# model_rf = pickle.load(open(rf_model_filepath, 'rb'))

# Model (IV) - Neural Network Model

In [23]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [24]:
# define neural network model
model_nn = Sequential()
model_nn.add(Dense(12, input_dim=16, kernel_initializer='random_uniform', activation='relu'))
model_nn.add(Dense(8, kernel_initializer='random_uniform', activation='relu'))
model_nn.add(Dense(4, kernel_initializer='random_uniform', activation='relu'))
model_nn.add(Dense(1, kernel_initializer='random_uniform', activation='relu'))

# compile model
model_nn.compile(loss='mse', optimizer='adam')

In [25]:
model_nn.fit(X_train, y_train, batch_size=300, epochs=10000, 
             verbose=1, 
             callbacks=[EarlyStopping(monitor='loss', min_delta=1000, patience=300)])

Epoch 1/10000
4374/4374 [==============================] - 0s 53us/step - loss: 3776193.2462
Epoch 2/10000
4374/4374 [==============================] - 0s 4us/step - loss: 3735924.1029
Epoch 3/10000
4374/4374 [==============================] - 0s 4us/step - loss: 3566646.2534
Epoch 4/10000
4374/4374 [==============================] - 0s 4us/step - loss: 3046852.9707
Epoch 5/10000
4374/4374 [==============================] - 0s 4us/step - loss: 2044744.1204
Epoch 6/10000
4374/4374 [==============================] - 0s 4us/step - loss: 1306576.3735
Epoch 7/10000
4374/4374 [==============================] - 0s 4us/step - loss: 1286138.8483
Epoch 8/10000
4374/4374 [==============================] - 0s 5us/step - loss: 1257355.1751
Epoch 9/10000
4374/4374 [==============================] - 0s 5us/step - loss: 1251241.9928
Epoch 10/10000
4374/4374 [==============================] - 0s 4us/step - loss: 1247212.3308
Epoch 11/10000
4374/4374 [==============================] - 0s 4us/step - loss

4374/4374 [==============================] - 0s 4us/step - loss: 805801.4932
Epoch 90/10000
4374/4374 [==============================] - 0s 4us/step - loss: 782553.9896
Epoch 91/10000
4374/4374 [==============================] - 0s 4us/step - loss: 752810.3810
Epoch 92/10000
4374/4374 [==============================] - 0s 4us/step - loss: 728819.5782
Epoch 93/10000
4374/4374 [==============================] - 0s 4us/step - loss: 705429.9744
Epoch 94/10000
4374/4374 [==============================] - 0s 4us/step - loss: 681069.8624
Epoch 95/10000
4374/4374 [==============================] - 0s 4us/step - loss: 660670.4952
Epoch 96/10000
4374/4374 [==============================] - 0s 4us/step - loss: 640498.2470
Epoch 97/10000
4374/4374 [==============================] - 0s 4us/step - loss: 623264.6755
Epoch 98/10000
4374/4374 [==============================] - 0s 4us/step - loss: 619414.8756
Epoch 99/10000
4374/4374 [==============================] - 0s 4us/step - loss: 596710.1683
Epo

4374/4374 [==============================] - 0s 4us/step - loss: 301257.5629
Epoch 178/10000
4374/4374 [==============================] - 0s 4us/step - loss: 304844.8466
Epoch 179/10000
4374/4374 [==============================] - 0s 4us/step - loss: 303178.2923
Epoch 180/10000
4374/4374 [==============================] - 0s 4us/step - loss: 301175.5976
Epoch 181/10000
4374/4374 [==============================] - 0s 4us/step - loss: 301847.0573
Epoch 182/10000
4374/4374 [==============================] - 0s 5us/step - loss: 308666.0490
Epoch 183/10000
4374/4374 [==============================] - 0s 4us/step - loss: 303569.9150
Epoch 184/10000
4374/4374 [==============================] - 0s 4us/step - loss: 302949.4532
Epoch 185/10000
4374/4374 [==============================] - 0s 4us/step - loss: 297435.9535
Epoch 186/10000
4374/4374 [==============================] - 0s 5us/step - loss: 302463.3975
Epoch 187/10000
4374/4374 [==============================] - 0s 5us/step - loss: 29917

4374/4374 [==============================] - 0s 5us/step - loss: 275037.7980
Epoch 266/10000
4374/4374 [==============================] - 0s 5us/step - loss: 265524.5182
Epoch 267/10000
4374/4374 [==============================] - 0s 5us/step - loss: 266097.3722
Epoch 268/10000
4374/4374 [==============================] - 0s 5us/step - loss: 266239.8216
Epoch 269/10000
4374/4374 [==============================] - 0s 5us/step - loss: 264673.1952
Epoch 270/10000
4374/4374 [==============================] - 0s 5us/step - loss: 261656.0920
Epoch 271/10000
4374/4374 [==============================] - 0s 5us/step - loss: 261205.8754
Epoch 272/10000
4374/4374 [==============================] - 0s 5us/step - loss: 267519.3000
Epoch 273/10000
4374/4374 [==============================] - 0s 5us/step - loss: 267541.5302
Epoch 274/10000
4374/4374 [==============================] - 0s 5us/step - loss: 270300.5296
Epoch 275/10000
4374/4374 [==============================] - 0s 5us/step - loss: 26914

4374/4374 [==============================] - 0s 5us/step - loss: 239457.4600
Epoch 354/10000
4374/4374 [==============================] - 0s 5us/step - loss: 238260.2624
Epoch 355/10000
4374/4374 [==============================] - 0s 5us/step - loss: 238501.3583
Epoch 356/10000
4374/4374 [==============================] - 0s 5us/step - loss: 237667.9820
Epoch 357/10000
4374/4374 [==============================] - 0s 5us/step - loss: 240374.4103
Epoch 358/10000
4374/4374 [==============================] - 0s 5us/step - loss: 246833.8152
Epoch 359/10000
4374/4374 [==============================] - 0s 5us/step - loss: 258144.2398
Epoch 360/10000
4374/4374 [==============================] - 0s 4us/step - loss: 247852.7236
Epoch 361/10000
4374/4374 [==============================] - 0s 5us/step - loss: 243986.1215
Epoch 362/10000
4374/4374 [==============================] - 0s 5us/step - loss: 238087.2634
Epoch 363/10000
4374/4374 [==============================] - 0s 5us/step - loss: 23773

...

4374/4374 [==============================] - 0s 5us/step - loss: 167815.6083
Epoch 3754/10000
4374/4374 [==============================] - 0s 6us/step - loss: 177416.9434
Epoch 3755/10000
4374/4374 [==============================] - 0s 5us/step - loss: 193250.5587
Epoch 3756/10000
4374/4374 [==============================] - 0s 4us/step - loss: 188585.0807
Epoch 3757/10000
4374/4374 [==============================] - 0s 5us/step - loss: 181065.5116
Epoch 3758/10000
4374/4374 [==============================] - 0s 5us/step - loss: 185914.0607
Epoch 3759/10000
4374/4374 [==============================] - 0s 5us/step - loss: 178345.6176
Epoch 3760/10000
4374/4374 [==============================] - 0s 5us/step - loss: 177020.0696
Epoch 3761/10000
4374/4374 [==============================] - 0s 5us/step - loss: 175215.2931
Epoch 3762/10000
4374/4374 [==============================] - 0s 4us/step - loss: 174350.5884
Epoch 3763/10000
4374/4374 [==============================] - 0s 5us/step - l

4374/4374 [==============================] - 0s 5us/step - loss: 171931.3973
Epoch 3841/10000
4374/4374 [==============================] - 0s 6us/step - loss: 169014.4143
Epoch 3842/10000
4374/4374 [==============================] - 0s 5us/step - loss: 171637.4167
Epoch 3843/10000
4374/4374 [==============================] - 0s 4us/step - loss: 169338.3948
Epoch 3844/10000
4374/4374 [==============================] - 0s 5us/step - loss: 175798.5315
Epoch 3845/10000
4374/4374 [==============================] - 0s 5us/step - loss: 174145.7063
Epoch 3846/10000
4374/4374 [==============================] - 0s 5us/step - loss: 173035.3916
Epoch 3847/10000
4374/4374 [==============================] - 0s 6us/step - loss: 172537.2810
Epoch 3848/10000
4374/4374 [==============================] - 0s 6us/step - loss: 174548.5864
Epoch 3849/10000
4374/4374 [==============================] - 0s 5us/step - loss: 178921.4383
Epoch 3850/10000
4374/4374 [==============================] - 0s 5us/step - l

4374/4374 [==============================] - 0s 6us/step - loss: 172624.8751
Epoch 3928/10000
4374/4374 [==============================] - 0s 6us/step - loss: 170510.4874
Epoch 3929/10000
4374/4374 [==============================] - 0s 6us/step - loss: 168313.1475
Epoch 3930/10000
4374/4374 [==============================] - 0s 5us/step - loss: 177211.0496
Epoch 3931/10000
4374/4374 [==============================] - 0s 5us/step - loss: 177216.4439
Epoch 3932/10000
4374/4374 [==============================] - 0s 5us/step - loss: 172176.1559
Epoch 3933/10000
4374/4374 [==============================] - 0s 5us/step - loss: 181728.1429
Epoch 3934/10000
4374/4374 [==============================] - 0s 5us/step - loss: 169596.4042
Epoch 3935/10000
4374/4374 [==============================] - 0s 5us/step - loss: 172967.6167
Epoch 3936/10000
4374/4374 [==============================] - 0s 5us/step - loss: 167378.9025
Epoch 3937/10000
4374/4374 [==============================] - 0s 5us/step - l

4374/4374 [==============================] - 0s 5us/step - loss: 173854.7211
Epoch 4015/10000
4374/4374 [==============================] - 0s 5us/step - loss: 166419.7526
Epoch 4016/10000
4374/4374 [==============================] - 0s 6us/step - loss: 175942.8797
Epoch 4017/10000
4374/4374 [==============================] - 0s 5us/step - loss: 172215.2924
Epoch 4018/10000
4374/4374 [==============================] - 0s 5us/step - loss: 182895.5158
Epoch 4019/10000
4374/4374 [==============================] - 0s 6us/step - loss: 186726.6336
Epoch 4020/10000
4374/4374 [==============================] - 0s 5us/step - loss: 179432.9502
Epoch 4021/10000
4374/4374 [==============================] - 0s 6us/step - loss: 167478.1436
Epoch 4022/10000
4374/4374 [==============================] - 0s 6us/step - loss: 170698.5727
Epoch 4023/10000
4374/4374 [==============================] - 0s 5us/step - loss: 168056.2681
Epoch 4024/10000
4374/4374 [==============================] - 0s 5us/step - l

4374/4374 [==============================] - 0s 4us/step - loss: 163951.0943
Epoch 4102/10000
4374/4374 [==============================] - 0s 4us/step - loss: 178202.3648
Epoch 4103/10000
4374/4374 [==============================] - 0s 5us/step - loss: 165143.7646
Epoch 4104/10000
4374/4374 [==============================] - 0s 5us/step - loss: 168535.0754
Epoch 4105/10000
4374/4374 [==============================] - 0s 5us/step - loss: 165489.3098
Epoch 4106/10000
4374/4374 [==============================] - 0s 5us/step - loss: 171681.8252
Epoch 4107/10000
4374/4374 [==============================] - 0s 4us/step - loss: 170949.4793
Epoch 4108/10000
4374/4374 [==============================] - 0s 5us/step - loss: 168741.4918
Epoch 4109/10000
4374/4374 [==============================] - 0s 4us/step - loss: 169261.8479
Epoch 4110/10000
4374/4374 [==============================] - 0s 4us/step - loss: 165117.3861
Epoch 4111/10000
4374/4374 [==============================] - 0s 5us/step - l

In [26]:
# nn_model_filepath = "/Users/apple/Documents/Data Projects/Machine_Learning_Apartment_Rent/static/resources/NN_Model.h5"
# model_nn.save(nn_model_filepath)

# Model (V) - K Nearest Neighbour

In [27]:
from sklearn.neighbors import KNeighborsRegressor
k_neighbour = KNeighborsRegressor()

from sklearn.model_selection import GridSearchCV
param_grid = {'n_neighbors' : list(range(1,999,2))}
model_k_neighbour = GridSearchCV(k_neighbour, param_grid, verbose=3)

model_k_neighbour.fit(X_train, y_train)

model_k_neighbour.best_params_

model_k_neighbour.best_score_

Fitting 3 folds for each of 499 candidates, totalling 1497 fits
[CV] n_neighbors=1 ...................................................
[CV] ........... n_neighbors=1, score=0.911738041980998, total=   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......... n_neighbors=1, score=0.8601791831498506, total=   0.0s
[CV] n_neighbors=1 ...................................................
[CV] .......... n_neighbors=1, score=0.9177267854641082, total=   0.0s
[CV] n_neighbors=3 ...................................................
[CV] .......... n_neighbors=3, score=0.9180071776410968, total=   0.0s
[CV] n_neighbors=3 ...................................................
[CV] ........... n_neighbors=3, score=0.876044709779677, total=   0.0s
[CV] n_neighbors=3 ...................................................


/Users/apple/Documents/anaconda3/envs/PyData/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV] .......... n_neighbors=3, score=0.9262704478831104, total=   0.0s
[CV] n_neighbors=5 ...................................................
[CV] .......... n_neighbors=5, score=0.8776286051303448, total=   0.0s
[CV] n_neighbors=5 ...................................................
[CV] .......... n_neighbors=5, score=0.8209184508842001, total=   0.0s
[CV] n_neighbors=5 ...................................................
[CV] .......... n_neighbors=5, score=0.9187974368421963, total=   0.0s
[CV] n_neighbors=7 ...................................................
[CV] .......... n_neighbors=7, score=0.8375805502793469, total=   0.0s
[CV] n_neighbors=7 ...................................................
[CV] .......... n_neighbors=7, score=0.7819873232476976, total=   0.0s
[CV] n_neighbors=7 ...................................................
[CV] .......... n_neighbors=7, score=0.8984476383648018, total=   0.0s
[CV] n_neighbors=9 ...................................................
[CV] .

[CV] ......... n_neighbors=43, score=0.5022151279477297, total=   0.0s
[CV] n_neighbors=45 ..................................................
[CV] ........ n_neighbors=45, score=0.41286676850741555, total=   0.0s
[CV] n_neighbors=45 ..................................................
[CV] ......... n_neighbors=45, score=0.5283238966008523, total=   0.0s
[CV] n_neighbors=45 ..................................................
[CV] ........ n_neighbors=45, score=0.49549092813147694, total=   0.0s
[CV] n_neighbors=47 ..................................................
[CV] ......... n_neighbors=47, score=0.4036283552157085, total=   0.0s
[CV] n_neighbors=47 ..................................................
[CV] ......... n_neighbors=47, score=0.5094883996339516, total=   0.0s
[CV] n_neighbors=47 ..................................................
[CV] ......... n_neighbors=47, score=0.4875705517551163, total=   0.0s
[CV] n_neighbors=49 ..................................................
[CV] .

[CV] ......... n_neighbors=83, score=0.4167063750431561, total=   0.0s
[CV] n_neighbors=85 ..................................................
[CV] ......... n_neighbors=85, score=0.3316255195898353, total=   0.0s
[CV] n_neighbors=85 ..................................................
[CV] ......... n_neighbors=85, score=0.4257354659829803, total=   0.0s
[CV] n_neighbors=85 ..................................................
[CV] .......... n_neighbors=85, score=0.412247079547601, total=   0.0s
[CV] n_neighbors=87 ..................................................
[CV] ........ n_neighbors=87, score=0.33033779965183785, total=   0.0s
[CV] n_neighbors=87 ..................................................
[CV] ......... n_neighbors=87, score=0.4233170880792143, total=   0.0s
[CV] n_neighbors=87 ..................................................
[CV] ......... n_neighbors=87, score=0.4070299086195669, total=   0.0s
[CV] n_neighbors=89 ..................................................
[CV] .

...

[CV] ....... n_neighbors=897, score=0.10500958235410096, total=   0.2s
[CV] n_neighbors=897 .................................................
[CV] ....... n_neighbors=897, score=0.17886630297302342, total=   0.2s
[CV] n_neighbors=897 .................................................
[CV] ....... n_neighbors=897, score=0.13452388317766373, total=   0.2s
[CV] n_neighbors=899 .................................................
[CV] ....... n_neighbors=899, score=0.10478853513484744, total=   0.2s
[CV] n_neighbors=899 .................................................
[CV] ........ n_neighbors=899, score=0.1785936706512956, total=   0.2s
[CV] n_neighbors=899 .................................................
[CV] ....... n_neighbors=899, score=0.13439919618274965, total=   0.2s
[CV] n_neighbors=901 .................................................
[CV] ........ n_neighbors=901, score=0.1045828466484602, total=   0.2s
[CV] n_neighbors=901 .................................................
[CV] .

[CV] ....... n_neighbors=935, score=0.17365675381787793, total=   0.2s
[CV] n_neighbors=935 .................................................
[CV] ........ n_neighbors=935, score=0.1313606136793668, total=   0.2s
[CV] n_neighbors=937 .................................................
[CV] ....... n_neighbors=937, score=0.10076527013151616, total=   0.2s
[CV] n_neighbors=937 .................................................
[CV] ....... n_neighbors=937, score=0.17344595902370108, total=   0.2s
[CV] n_neighbors=937 .................................................
[CV] ....... n_neighbors=937, score=0.13120241590973247, total=   0.2s
[CV] n_neighbors=939 .................................................
[CV] ....... n_neighbors=939, score=0.10054976476919808, total=   0.2s
[CV] n_neighbors=939 .................................................
[CV] ....... n_neighbors=939, score=0.17317755937571067, total=   0.2s
[CV] n_neighbors=939 .................................................
[CV] .

[CV] ....... n_neighbors=973, score=0.12838929417526979, total=   0.2s
[CV] n_neighbors=975 .................................................
[CV] ....... n_neighbors=975, score=0.09731858136020212, total=   0.3s
[CV] n_neighbors=975 .................................................
[CV] ....... n_neighbors=975, score=0.16805324050745285, total=   0.2s
[CV] n_neighbors=975 .................................................
[CV] ....... n_neighbors=975, score=0.12825734413256218, total=   0.2s
[CV] n_neighbors=977 .................................................
[CV] ......... n_neighbors=977, score=0.097101051087775, total=   0.2s
[CV] n_neighbors=977 .................................................
[CV] ....... n_neighbors=977, score=0.16779353168523692, total=   0.2s
[CV] n_neighbors=977 .................................................
[CV] ....... n_neighbors=977, score=0.12816287954167727, total=   0.2s
[CV] n_neighbors=979 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done 1497 out of 1497 | elapsed:  9.6min finished


0.9067741117679614

# Model Comparison

In [28]:
print(f"Linear Model R-quared (R2): {r2_score(y_test, lin_model.predict(X_test))}")
print(f"Polynomial Model R-quared (R2): {poly_model.score(X_test, y_test)}")
print(f"Random Forest Model R-quared (R2): {model_rf.best_score_}")
print(f"Neural Network Model R-quared (R2): {r2_score(y_test, model_nn.predict(X_test))}")
print(f"K Nearest Neighbour Model R-quared (R2): {model_k_neighbour.best_score_}")

Linear Model R-quared (R2): 0.5344241825987113
Polynomial Model R-quared (R2): 0.44883007398009794
Random Forest Model R-quared (R2): 0.9367381542339314
Neural Network Model R-quared (R2): 0.7489722404926968
K Nearest Neighbour Model R-quared (R2): 0.9067741117679614
